# NWB Tutorial - Reading NWB Data in Python

## Introduction

In this tutorial, we will read single neuron spiking data that is in the NWB standard format and do a basic visualization of the data.

## Download the data

First, let's download an NWB data file from the [DANDI neurophysiology data archive](https://dandiarchive.org/). 

An NWB file represents a single session of an experiment. It contains all the data of that session and the metadata required to understand the data. 

We will use data from one session of an experiment by [Chandravadia et al. (2020)](https://www.nature.com/articles/s41597-020-0415-9), where the authors recorded single neuron activity from the medial temporal lobes of human subjects while they performed a recognition memory task.

1. Go to the DANDI page for this dataset: https://dandiarchive.org/dandiset/000004/draft
2. Toward the top middle of the page, click the "View Data" button. 
<img src="images/dandi_download1.png" width="600">

3. Click on the folder "sub-P11MHM" (click the folder name, not the checkbox).
<img src="images/dandi_download2.png" width="600">

4. Then click on the download symbol to the right of the filename "sub-P11HMH_ses-20061101_ecephys+image.nwb" to download the data file (69 MB) to your computer.
<img src="images/dandi_download3.png" width="600">

## Installing PyNWB

To read data in the NWB format in Python, we recommend using the [PyNWB](https://pynwb.readthedocs.io/) package created by the NWB development team.

First, install PyNWB using `pip` or `conda`. You will need Python 3.5+ installed.
- `pip install -U pynwb`
- `conda install -c conda-forge pynwb` 

## Using NWBHDF5IO

To read and write data in NWB, we will use the `NWBHDF5IO` class. You can read this as "NWB" "HDF5" "IO". This class reads NWB data that is in the HDF5 storage format, a popular, hierarchical format for storing large-scale scientific data. "IO" stands for Input/Output which means reading/writing data.

In [2]:
from pynwb import NWBHDF5IO

# Change the string below to the path of the file on your computer
filepath = 'C:/Users/uggriffin/Documents/GitHub/NeuroDataReHack/nwb_tutorial-main/HCK13/sub-P11HMH_ses-20061101_ecephys+image.nwb'
io = NWBHDF5IO(filepath, 'r')  # open the file in read mode 'r'


Call the `NWBHDF5IO.read()` method to read the NWB data into an `NWBFile` object. Print the `NWBFile` object to inspect its contents.

In [3]:
nwb = io.read()
nwb

C:\Users\uggriffin\anaconda3\lib\site-packages\hdmf\utils.py:629: FutureWarning: DynamicTable.__init__: Using positional arguments for this method is discouraged and will be deprecated in a future major release. Please use keyword arguments to ensure future compatibility.
  warnings.warn(msg, FutureWarning)
C:\Users\uggriffin\anaconda3\lib\site-packages\hdmf\utils.py:629: FutureWarning: DynamicTable.__init__: Using positional arguments for this method is discouraged and will be deprecated in a future major release. Please use keyword arguments to ensure future compatibility.
  warnings.warn(msg, FutureWarning)


root pynwb.file.NWBFile at 0x2681564302352
Fields:
  acquisition: {
    events <class 'pynwb.misc.AnnotationSeries'>,
    experiment_ids <class 'pynwb.base.TimeSeries'>
  }
  data_collection: learning: 80, recognition: 81
  devices: {
    Neuralynx-cheetah <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    Neuralynx-cheetah-microwires-19 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Neuralynx-cheetah-microwires-2 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Neuralynx-cheetah-microwires-21 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Neuralynx-cheetah-microwires-22 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Neuralynx-cheetah-microwires-3 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Neuralynx-cheetah-microwires-4 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Neuralynx-cheetah-microwires-6 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Neuralynx-cheetah-microwires-7 <class 'pynwb.ecephys.ElectrodeGroup'>,
    Neuralynx-cheetah-microwires-8 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  elec

## Access stimulus data in NWB

Data representing stimuli that were presented to the experimental subject are stored in `NWBFile.stimulus`.

In [12]:
nwb.data_collection

'learning: 80, recognition: 81'

`NWBFile.stimulus` is a dictionary that can contain PyNWB objects representing different types of data, such as images or time series of images. In this file, `NWBFile.stimulus` contains a single key `'StimulusPresentation'` with an `OpticalSeries` object representing what images were shown to the subject and at what times.

In [13]:
stim = nwb.stimulus['StimulusPresentation']

## Lazy loading of datasets

Data arrays are read passively from the NWB file. Calling the `data` attribute on a `TimeSeries` such as an `OpticalSeries` does not read the data values, but presents an `h5py` object that can be indexed to read the data. You can use the `[:]` operator to read the entire data array into memory.

In [ ]:
stim = nwb.stimulus['StimulusPresentation']

print(stim.data.shape)
all_stim_data = stim.data[:]

Time series data in NWB can have multiple dimensions, and the first dimension always represents time. So this dataset has 200 images of size 400x300 pixels with three channels (red, green, and blue).

## Slicing datasets
It is often preferable to read only a portion of the data, e.g., because the full data array is too large to fit into your computer's RAM. To do this, index or slice into the `data` attribute just like if you were indexing or slicing a numpy array.

In [ ]:
pic_index = 31
img = stim.data[pic_index]

import matplotlib.pyplot as plt
img = img[...,::-1]  # reverse the last dimension because the data were stored in BGR instead of RGB
plt.imshow(img, aspect='auto')

## Access single unit data in NWB
Data and metadata about sorted single units are stored in `NWBFile.units`. `NWBFile.units` is a `Units` object that stores metadata about each single unit in a tabular form, where each row represents a unit and has spike times and additional metadata. Printing the `Units` object shows the column names and basic metadata associated with each unit.

In [ ]:
nwb.units

We can also view the single unit data as a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [ ]:
units_df = nwb.units.to_dataframe()
units_df

To access the spike times of the first single unit, index `nwb.units` with the column name 'spike_times' and then the row index, 0. All times in NWB are stored in seconds relative to the session start time.

In [ ]:
nwb.units['spike_times'][0]

## Visualize spiking activity relative to stimulus onset

Now we can look at when these single units spike relative to when image stimuli were presented to the subject.

Let's loop through the first 10 units and get their spike times. For each unit, loop through each stimulus onset time and compute the spike times relative to stimulus onset. Finally, create a raster plot and histogram of these aligned spike times.

In [ ]:
import numpy as np

before = 1.  # in seconds
after = 3.

stim_on_times = stim.timestamps[:]  # get the stimulus times for all stimuli

for unit in range(10):
    unit_spike_times = nwb.units['spike_times'][unit]
    trial_spikes = []
    for time in stim_on_times:
        # compute spike times relative to stimulus onset
        aligned_spikes = unit_spike_times - time
        # keep only spike times in a given time window around the stimulus onset
        aligned_spikes = aligned_spikes[(-before < aligned_spikes) & (aligned_spikes < after)]
        trial_spikes.append(aligned_spikes)
    fig, axs = plt.subplots(2, 1, sharex=True)
    plt.xlabel('time (s)')
    axs[0].eventplot(trial_spikes)
    
    axs[0].set_ylabel('trial')
    axs[0].set_title('unit {}'.format(unit))
    axs[0].axvline(0, color=[.5,.5,.5])
    
    axs[1].hist(np.hstack(trial_spikes), 30)
    axs[1].axvline(0, color=[.5,.5,.5])

## Accessing trial information in NWB

Trial data are stored in `NWBFile.trials`. `NWBFile.trials` is a `TimeIntervals` object that stores metadata about each trial in a tabular form, where each row represents a trial and has a start time, stop time, and additional metadata. Like for the `Units` table, you can also view the trial data as a pandas DataFrame.

In [ ]:
trials_df = nwb.trials.to_dataframe()
trials_df

The stimuli are stored in `nwb.stimuli` and can be mapped one to one to each row (trial) of `nwb.trials` based on the `stim_on_time` column. Let's visualize the first 10 images that were categorized as landscapes in the session. We can interact with the trial data as a PyNWB `TimeIntervals` type or as a pandas DataFrame. Here, let's use pandas functions.

In [ ]:
assert np.all(stim.timestamps[:] == trials_df.stim_on_time[:])

stim_on_times_landscapes = trials_df[trials_df.category_name == 'landscapes'].stim_on_time
for time in stim_on_times_landscapes[:10]:
    img = np.squeeze(stim.data[np.where(stim.timestamps[:] == time)])
    img = img[...,::-1]  # reverse the last dimension because the data were stored in BGR instead of RGB
    plt.figure()
    plt.imshow(img, aspect='auto')

## Exploring the NWB file

You can explore the NWB file by printing the `NWBFile` object and accessing its attributes, but it may be useful to explore the data in a less programmatic, more visual way. You can use NWBWidgets, a package containing interactive widgets for visualizing NWB data, or you can use the HDFView tool, which can open any generic HDF5 file, which an NWB file is.

### NWBWidgets

To use NWBWidgets, first install NWBWidgets:
- `pip install -U nwbwidgets`

Then import and run the `nwb2widget` function on the `NWBFile` object.

In [ ]:
from nwbwidgets import nwb2widget

nwb2widget(nwb)

### HDFView

To use HDFView to inspect and explore the NWB file, download and install HDFView from here: https://www.hdfgroup.org/downloads/hdfview/ and then open the NWB file using the program.

## Conclusion

This is an example of how to get started with understanding and analyzing public NWB datasets. This particular dataset was published with an extensive open analysis in both MATLAB and Python, which you can find [here](https://github.com/rutishauserlab/recogmem-release-NWB). For more datasets, or to publish your own NWB data for free, check out the DANDI archive. Also, make sure to check out the DANDI breakout session later in this event.

# Learn more!

## Python tutorials
### See our tutorials for more details about your data type:
* [Extracellular electrophysiology](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html#sphx-glr-tutorials-domain-ecephys-py)
* [Calcium imaging](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ophys.html#sphx-glr-tutorials-domain-ophys-py)
* [Intracellular electrophysiology](https://pynwb.readthedocs.io/en/stable/tutorials/domain/icephys.html#sphx-glr-tutorials-domain-icephys-py)

### Check out other tutorials that teach advanced NWB topics:
* [Iterative data write](https://pynwb.readthedocs.io/en/stable/tutorials/general/iterative_write.html#sphx-glr-tutorials-general-iterative-write-py)
* [Extensions](https://pynwb.readthedocs.io/en/stable/tutorials/general/extensions.html#sphx-glr-tutorials-general-extensions-py)
* [Advanced HDF5 I/O](https://pynwb.readthedocs.io/en/stable/tutorials/general/advanced_hdf5_io.html#sphx-glr-tutorials-general-advanced-hdf5-io-py)


## MATLAB tutorials
* [Extracellular electrophysiology](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/ecephys.html)
* [Calcium imaging](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/ophys.html)
* [Intracellular electrophysiology](https://neurodatawithoutborders.github.io/matnwb/tutorials/html/icephys.html)